## request 라이브러리

웹 페이지를 다운로드하기 위한 라이브러리

https://2.python-requests.org/en/master/user/quickstart/

In [ ]:
import requests

page = requests.get("https://dataquestio.github.io/web-scraping-pages/simple.html")
page

In [ ]:
# status_code 의미
# 2xx: 성공
# 4xx: 클라이언트 에러
# 5xx: 서버 에러
page.status_code

In [ ]:
# binary response content
page.content

## BeautifulSoup 라이브러리

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
print(soup.prettify())

In [ ]:
list(soup.children)

In [ ]:
# Doctype: 문서 타입에 대한 정보
# NavigableString: HTML 문서에서 찾아낸 텍스트
# Tag: 중첩(nested) 태그
[type(item) for item in list(soup.children)]

In [ ]:
# html 태그 안의 자식에 대한 결과
html = list(soup.children)[2]
list(html.children)

In [ ]:
# body 태그 안 자식에 대한 결과
body = list(html.children)[3]
list(body.children)

In [ ]:
# p 태그 안 텍스트
p = list(body.children)[1]
p.get_text()

## 태그명으로 태그 검색

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('p')

In [ ]:
# find_all은 list를 리턴함
# 따라서 첫 번째 텍스트를 얻기 위해 다음과 같이 함
soup.find_all('p')[0].get_text()

In [ ]:
# 첫 번째 인스턴스만 찾기 원할 경우 find 이용
soup.find('p')

## class와 id로 태그 검색

In [ ]:
page = requests.get("https://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content, 'html.parser')
soup

In [ ]:
soup.find_all('p', class_='outer-text')

In [ ]:
soup.find_all(class_="outer-text")

## CSS 선택자

p a — finds all a tags inside of a p tag.

body p a — finds all a tags inside of a p tag inside of a body tag.

html body — finds all body tags inside of an html tag.

p.outer-text — finds all p tags with a class of outer-text.

p#first — finds all p tags with an id of first.

body p.outer-text — finds any p tags with a class of outer-text inside of a body tag.

In [ ]:
soup.select("div p")

## 날씨 데이터 스크래핑


https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168

날씨 예보 웹 페이지 다운로드

웹 페이지 파싱하기 위한 BeautifulSoup 객체 생성

id가 seven-day-forecast인 div 태그 찾고 이를 seven_day에 할당

seven_day 내부에서 예보 아이템 찾기

첫 번째 예보 아이템 추출 및 출력

In [ ]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

tonight 변수 안의 정보 추출

예보 아이템 이름: Tonight

예보 설명: img 태그 안의 title 속성

짧은 예보 설명: 위 경우에는 Mostly Clear임

최저 기온: 위 경우에는 47도

In [ ]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

In [ ]:
img = tonight.find("img")
desc = img['title']
print(desc)

## 모든 정보 추출하기

변수 seven_day에서 class가 tombstone-container인 아이템 내부에서 class가 period-name인 모든 아이템을 선택

get_text 메소드 호출하여 리스트 생성

In [ ]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

In [ ]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

In [ ]:
import pandas as pd
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc":descs
})
weather

In [ ]:
temp_nums = weather["temp"].str.extract(r'(?P<digit>\d+)', expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

In [ ]:
weather["temp_num"].mean()

In [ ]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

In [ ]:
weather[is_night]